# PCA - SCM Classifier
This notebook shows how to train a Support Vector Machine classifier on the grayscale Rock-Paper-Scissors images using Principal Component Analysis for dimensionality reduction and Bayesian optimization of the model hyperparameters.

The model is trained using 75% of the dataset with a five fold cross-validation and achieves a f1 (micro) score of 97.6% on a test set of 547 images (the remainig 25% of the dataset)

**Author**: Julien de la Bruère-Terreault <drgfreeman@tuta.io>  
**Source**: https://github.com/DrGFreeman/rps-cv-data-science  
**License**: MIT

### Import packages
In addition to the common Numpy, and Scikit-Learn packages, this model also requires the Scikit-Optimize package for optimization and the rpscv package for preprocessing of the images. The instructions to install these packages are given in the README of the [GitHub repository](https://github.com/DrGFreeman/rps-cv-data-science).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from skopt import BayesSearchCV
from skopt.space import Real, Integer

from rpscv import imgproc

### Generate feature and labels from images
The `generateGrayFeatures` function of the `rpscv.imgproc` module reads the images, removes the green background, converts them to grayscale and rescales their values in the range of 0 (black) to 1 (white). The function outputs an array where each row represents the 60,000 pixel values of an image (200x300 pixels flattened) and a vector of label values in the form of integers (0 = *rock*, 1 = *paper*, 2 = *scissors*).

In [2]:
X, y = imgproc.generateGrayFeatures()

Completed processing 2188 images


### Train-test split
We split the dataset into training and testing sets using 75% of the images for training and 25% for testing.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25,
                                                    stratify=y, random_state=42)

### Model definition
The model consists of a Principal Component Analysis (PCA) transformer followed by a Support Vector Machine classifier. The PCA transformer will reduce the number of features from 60,000 (the number of pixels in the original images) to a reduced number of values. For more information on the PCA transformation, see the [PCA visualization](pca_visualization.ipynb) notebook in this repository.

The SVC classifier will take the transformed features from the PCA, along with the image labels, to fit a classification model and perform the predictions.

The PCA transformer and SVC classifier are assembled into a Scikit-Learn `Pipeline` object. This will allow to perform cross-validation on the training data during the model hyperparameter tuning without leaking data from the hold-out fold into the model.

The `rbf` kernel is selected for the support vector classifier as it allows more complex, non-linear decision boundaries.

In [4]:
pipeline = Pipeline([('pca', PCA()),
                     ('clf', SVC(kernel='rbf'))])

Next, we define the range of the model hyperparameters into which to perform the search. Since we will be using the `BayesSearchCV` class of the Scikit-Optimize package, we define the hyperparameter ranges using the `Integer` and `Real` classes of the same package.

We define the range of principal components (`n_components`) for the `PCA` transformer as integers between 20 and 100.

We define the `gamma` and `C` parameters of the `SVC` as reals with the ranges .0001-.01 and 1-3000 respectively.

In [5]:
opt_params = {'pca__n_components': Integer(20, 100),
              'clf__gamma': Real(.0001, .01, prior='log-uniform'),
              'clf__C': Real(1, 3000, prior='log-uniform')}

We define the model as the `BayesSearchCV` cross-validation object, passing the `pipeline` object defined previously as estimator. We pass the hyperparameter dictionary as search space and specify 100 iterations for the hyperparameter search.

We use 5 fold stratified cross-validation which means that at each search iteration, the training set will be split in five and the model fitted to 4 of the folds (80% of the training data) and evaluated on the remaining 20% of the training data. This will be repeated 5 times, changing the validation set each time, and the average score at each iteration will be kept. The use of `StratifiedKFold` ensures that each class will be equally represented in each of the cross-validation folds.

The scoring function used is the F1 score calculated globally (micro option). This metric balances the precision and recall taking into account class imbalance if present.

In [6]:
model = BayesSearchCV(pipeline,
                      search_spaces=opt_params,
                      n_iter=100,
                      cv=StratifiedKFold(n_splits=5),
                      scoring='f1_micro',
                      n_jobs=-1,
                      return_train_score=True)

### Fit model and evaluate on training set
With the model defined, we call the `fit` method on the model which performs the hyperparameter optimization.

In [7]:
%%time
model.fit(X_train, y_train)

CPU times: user 6min 22s, sys: 9min 25s, total: 15min 48s
Wall time: 35min 25s


BayesSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_iter=100, n_jobs=-1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=True, scoring='f1_micro',
       search_spaces={'pca__n_components': Integer(low=20, high=100), 'clf__gamma': Real(low=0.0001, high=0.01, prior='log-uniform', transform='identity'), 'clf__C': Real(low=1, high=3000, prior='log-uniform', transform='identity')},
       verbose=0)

We print the best set of hyperparamters found during the optimization as well as the model F1 score obtained on the training set with the k-fold cross-validation.

In [8]:
model.best_params_

{'clf__C': 289.6483653005734,
 'clf__gamma': 0.0006175507672838874,
 'pca__n_components': 67}

In [9]:
model.best_score_

0.9847653869591713

### Model evaluation on test set
By default, the `BayesSearchCV` object, as well as the Scikit-Learn `GridSearchCV` of `RandomSearchCV` objects, retrain the model, with the best hyperparameters, on the full training set at the end of the hyperparameter optimization. We therefore don't need to perform this test manually prior evaluating the model on the test set.

We use the model `score` method to evaluate the model F1 score on the test set which it has never seen.

In [10]:
model.score(X_test, y_test)

0.9762340036563071

The model achieves a score of 0.976 which is excellent for a "traditional" (i.e. non deep-learning) model on such a small dataset with no data augmentation used. There is also only a small loss of performance compared to the model score on the training set which indicates that the model is not overfitting the data.

To further evaluate the model performance, we use the model's `predict` method to predict the labels of the test set.

In [11]:
y_pred = model.predict(X_test)

For convenience, we import the `gestureTxt` dictionary from the `rpscv.utils` module. This dictionary provides the text labels matching the integer values returned by the `generateGrayFeatures` function.

In [12]:
from rpscv.utils import gestureTxt

We print the classification report using the predicted labels `y_pred` and the true labels `y_test`. This report provides the precision, recall and f1-score values for each class as well as the average values for each score using different weightings.

In [13]:
print(classification_report(y_test, y_pred, target_names=gestureTxt.values()))

              precision    recall  f1-score   support

        rock       0.96      0.99      0.98       182
       paper       0.97      0.97      0.97       178
    scissors       1.00      0.97      0.99       187

   micro avg       0.98      0.98      0.98       547
   macro avg       0.98      0.98      0.98       547
weighted avg       0.98      0.98      0.98       547



Finally, we print the confusion matrix of predictions on the test set which helps undertand where the models predictions are wrong.

For instance, looking at the columns, we can see that the models predicted *rock* for six of the *paper* images and one *scissors* image, which explains the lower precision score of 96% of the *rock* class reported in the classification report. In opposition, all predicted *scissors* are indeed *scissors*, explaining the 100% precision score on the *scissors* class.

Looking at the rows, we can see that out of the 182 *rock* images, only two we incorrectly predicted as *paper*. This shows in the 99% recall score for the *rock* class in the classification report compared to the 97% recall scores of the *paper* and *scissors* classes which have mode incorrectly predicted images.

In [14]:
conf_matrix = pd.DataFrame(confusion_matrix(y_test, y_pred))
conf_matrix.index = pd.MultiIndex.from_tuples([('true label', label) for label in gestureTxt.values()])
conf_matrix.columns = pd.MultiIndex.from_tuples([('predicted label', label) for label in gestureTxt.values()])
conf_matrix

predicted label               
                               rock paper scissors
true label rock                 180     2        0
           paper                  6   172        0
           scissors               1     4      182